In [ ]:
!pip install pandas
!pip install ace_tools

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem, MolFromSmiles
from rdkit.Chem.SaltRemover import SaltRemover
import numpy as np
import pandas as pd
from mordred import Calculator, descriptors
import logging
import os

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def t3d(mol):
    """
    Convert a molecule to a 3D structure.
    Returns None if conversion fails.
    """
    try:
        if mol is None:
            return None  # Return None for invalid molecules

        mol_3D = Chem.AddHs(mol)

        # Embed the 3D structure
        if AllChem.EmbedMolecule(mol_3D, maxAttempts=5000) != 0:
            logger.warning("Failed to embed molecule")
            return None  # Embedding failed

        AllChem.UFFOptimizeMolecule(mol_3D)

        # Compute Gasteiger charges
        AllChem.ComputeGasteigerCharges(mol_3D)

        return mol_3D
    except Exception as e:
        logger.error(f'Error processing molecule: {e}')
        return None

def main():
    # Input and output file paths
    input_csv = 'cancer_drugs_smiles_from_chembl_pubchem.csv'
    output_csv = 'cancer_smiles_descriptors.csv'
    
    logger.info(f"Loading data from {input_csv}")
    
    # Check if input file exists
    if not os.path.exists(input_csv):
        logger.error(f"Input file {input_csv} not found")
        return
    
    # Load dataset
    df = pd.read_csv(input_csv)
    
    # Initialize salt remover
    remover = SaltRemover()
    
    # Create lists to store results
    valid_mols = []
    drug_names = []
    smiles_strings = []
    chembl_ids = []
    
    # Count total molecules
    total_mols = len(df)
    logger.info(f"Processing {total_mols} molecules")
    
    # Process each molecule
    for i, row in df.iterrows():
        drug_name = row.get('Drug Name', '')
        smile = row.get('SMILES', '')
        chembl_id = row.get('ChEMBL ID', '')
        
        # Skip rows with no SMILES
        if pd.isna(smile) or smile == '':
            logger.info(f"Skipping drug {drug_name} - no SMILES provided")
            continue
            
        # Convert SMILES to molecule
        mol = MolFromSmiles(smile)
        if mol:
            # Strip salts
            stripped_mol = remover.StripMol(mol)
            # Convert to 3D
            mol_3D = t3d(stripped_mol)
            if mol_3D:
                valid_mols.append(mol_3D)
                drug_names.append(drug_name)
                smiles_strings.append(smile)
                chembl_ids.append(chembl_id)
                if (i + 1) % 10 == 0:
                    logger.info(f"Processed {i+1} of {total_mols} molecules")
            else:
                logger.warning(f"Failed to generate 3D structure for {drug_name}")
        else:
            logger.warning(f"Failed to parse SMILES for {drug_name}: {smile}")
    
    logger.info(f"Successfully processed {len(valid_mols)} valid molecules out of {total_mols}")
    
    # Calculate descriptors
    logger.info("Calculating descriptors...")
    try:
        calc = Calculator(descriptors, ignore_3D=False)
        descriptor_df = calc.pandas(valid_mols)
        
        # Add drug information to descriptor dataframe
        descriptor_df['Drug Name'] = drug_names
        descriptor_df['SMILES'] = smiles_strings
        descriptor_df['ChEMBL ID'] = chembl_ids
        
        # Reorder columns to have drug info first
        cols = descriptor_df.columns.tolist()
        info_cols = ['Drug Name', 'SMILES', 'ChEMBL ID']
        descriptor_cols = [col for col in cols if col not in info_cols]
        descriptor_df = descriptor_df[info_cols + descriptor_cols]
        
        # Save to CSV
        logger.info(f"Saving results to {output_csv}")
        descriptor_df.to_csv(output_csv, index=False)
        logger.info(f"Successfully saved {len(descriptor_df)} drugs with descriptors to {output_csv}")
        
        # Print a summary
        logger.info(f"Descriptor calculation summary:")
        logger.info(f"  Total drugs in input: {total_mols}")
        logger.info(f"  Drugs with valid SMILES and 3D structures: {len(valid_mols)}")
        logger.info(f"  Number of descriptors calculated: {len(descriptor_cols)}")
        
    except Exception as e:
        logger.error(f"Error calculating descriptors: {e}")

if __name__ == "__main__":
    main()

2025-04-20 04:29:36,511 - INFO - Loading data from cancer_drugs_smiles_from_chembl_pubchem.csv
2025-04-20 04:29:36,530 - INFO - Processing 321 molecules
2025-04-20 04:29:38,004 - INFO - Skipping drug Afamitresgene Autoleucel - no SMILES provided
2025-04-20 04:29:38,124 - INFO - Skipping drug Aflibercept - no SMILES provided
2025-04-20 04:29:38,125 - INFO - Skipping drug Aldesleukin - no SMILES provided
2025-04-20 04:29:38,318 - INFO - Processed 10 of 321 molecules
2025-04-20 04:29:38,319 - INFO - Skipping drug Alemtuzumab - no SMILES provided
2025-04-20 04:29:38,547 - INFO - Skipping drug Amivantamab - no SMILES provided
2025-04-20 04:29:38,873 - INFO - Processed 20 of 321 molecules
[04:29:38] UFFTYPER: Unrecognized charge state for atom: 0
[04:29:38] UFFTYPER: Unrecognized atom type: As1+3 (0)
[04:29:38] UFFTYPER: Unrecognized charge state for atom: 0
[04:29:38] UFFTYPER: Unrecognized atom type: As1+3 (0)
[04:29:38] UFFTYPER: Unrecognized charge state for atom: 0
[04:29:38] UFFTYPER: 

In [1]:
smiley_biodata_csv_path = "cancer_smiles_descriptors.csv"
output_csv = "cancer_filtered_biodata_data.csv"
cross_mapped_csv = "cancer_final_data.csv"

STR_allowed_threshold = 0.20
replace_String_with = 'median'
Smiles_column_name = 'SMILES'




neuro_smiley_csv = "cancer_drugs_target_smiles.csv"
neuro_smile_col = "SMILES"
neuro_target_col = "Targets"
neuro_drug_name_col = "Product"

In [4]:
def improove_raw_data(raw_file_path, output_csv_path):
    smiley_biodata_data = pd.read_csv(raw_file_path)
    smiley_biodata_data[Smiles_column_name] = smiley_biodata_data[Smiles_column_name].str.lstrip('\n')
    smiley_biodata_data = smiley_biodata_data.drop_duplicates(subset=Smiles_column_name, keep='first')
    cols = smiley_biodata_data.columns
    val_counts = {}
    for col in cols:
        val_counts[col] = smiley_biodata_data[col].apply(lambda x: type(x)).value_counts()
    str_cols = []
    for key in val_counts.keys():
        if str in val_counts[key].keys():
            str_cols.append(key)
    if Smiles_column_name in str_cols:
        str_cols.remove(Smiles_column_name)
        
        
    for str_col in str_cols:
        initial_Data = smiley_biodata_data[str_col]
        final_Data = []
        for i in initial_Data:
            n = np.nan
            try:
                n = float(i)
            except:
                n = np.nan
            final_Data.append(n)
        smiley_biodata_data[str_col] = final_Data
    
    nan_cols = smiley_biodata_data.columns[smiley_biodata_data.isna().any()].tolist()
    nan_counts = {}
    for nan_col in nan_cols:
        n = smiley_biodata_data[nan_col].isna().sum()
        if n in nan_counts.keys():
            nan_counts[n].append(nan_col)
        else:
            nan_counts[n] = [nan_col]
    # sort nan_counts
    nan_counts = dict(sorted(nan_counts.items()))
    n_data = smiley_biodata_data.shape[0]
    str_allowed = int(n_data * STR_allowed_threshold)
    str_cols_to_drop = []
    for x,y in nan_counts.items():
        if x > str_allowed:
            for col in y:
                str_cols_to_drop.append(col)
                
                
    print(f"Droping {len(str_cols_to_drop)} columns out of {len(str_cols)} string columns out of {len(cols)} total columns with more than {STR_allowed_threshold*100}% NaN values")
    smiley_biodata_data = smiley_biodata_data.drop(columns=str_cols_to_drop)
    
    count = 0
    for col in smiley_biodata_data.columns:
        if smiley_biodata_data[col].isna().sum() > 0:
            count += 1
            # print(col)
            median = smiley_biodata_data[col].median()
            smiley_biodata_data[col] = smiley_biodata_data[col].fillna(median)
            
    print(f"Filled {count} columns with median values")
    smiley_biodata_data.to_csv(output_csv, index=False)
    

improove_raw_data(smiley_biodata_csv_path, output_csv)

Droping 185 columns out of 1087 string columns out of 1829 total columns with more than 20.0% NaN values
Filled 902 columns with median values


In [5]:
neuro_smiley_data = pd.read_csv(neuro_smiley_csv)
filtered_smiley_data = pd.read_csv(output_csv)
neuro_cols = list(neuro_smiley_data.columns)
print(neuro_cols, type(neuro_cols))
neuro_cols.remove(neuro_smile_col)
neuro_cols.remove(neuro_drug_name_col)
# print(neuro_target_col)
neuro_cols.remove(neuro_target_col)
neuro_smiley_data = neuro_smiley_data.drop(columns=neuro_cols)
# neuro_smiley_data.head()

['Product', 'Targets', 'SMILES'] <class 'list'>


In [6]:
# lstrip \n from smile column
neuro_smiley_data[neuro_smile_col] = neuro_smiley_data[neuro_smile_col].str.lstrip('\n')

# upper case all drugs and targets
neuro_smiley_data[neuro_drug_name_col] = neuro_smiley_data[neuro_drug_name_col].str.upper()
# filtered_smiley_data
neuro_smiley_data[neuro_target_col] = neuro_smiley_data[neuro_target_col].str.upper()
# neuro_smiley_data.head()


In [7]:
# split targets with ",", if 2 values in target are sep by ";", then split them into 2 rows
neuro_smiley_data[neuro_target_col] = neuro_smiley_data[neuro_target_col].str.split(';')
neuro_smiley_data = neuro_smiley_data.explode(neuro_target_col)
neuro_smiley_data
# neuro_smiley_data = neuro_smiley_data.dropna()
# drop 'nan' strings

# neuro_smiley_data.shape

,Product,Targets,SMILES
0,ABEMACICLIB,CDKN2A,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,PGR,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,CDK4,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,ERBB2,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,CDK6,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
...,...,...,...
229,VORINOSTAT,TUBA4A,O=C(CCCCCCC(=O)Nc1ccccc1)NO
230,ZANUBRUTINIB,TP53,C=CC(=O)N1CCC([C@@H]2CCNc3c(C(N)=O)c(-c4ccc(Oc...
230,ZANUBRUTINIB,BTK,C=CC(=O)N1CCC([C@@H]2CCNc3c(C(N)=O)c(-c4ccc(Oc...
230,ZANUBRUTINIB,EFNB2,C=CC(=O)N1CCC([C@@H]2CCNc3c(C(N)=O)c(-c4ccc(Oc...


In [9]:
len(neuro_smiley_data["Targets"].unique())

1589

In [15]:
neuro_smiley_data

,Product,Targets,SMILES
0,ABEMACICLIB,CDKN2A,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,PGR,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,CDK4,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,ERBB2,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
0,ABEMACICLIB,CDK6,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...
...,...,...,...
229,VORINOSTAT,TUBA4A,O=C(CCCCCCC(=O)Nc1ccccc1)NO
230,ZANUBRUTINIB,TP53,C=CC(=O)N1CCC([C@@H]2CCNc3c(C(N)=O)c(-c4ccc(Oc...
230,ZANUBRUTINIB,BTK,C=CC(=O)N1CCC([C@@H]2CCNc3c(C(N)=O)c(-c4ccc(Oc...
230,ZANUBRUTINIB,EFNB2,C=CC(=O)N1CCC([C@@H]2CCNc3c(C(N)=O)c(-c4ccc(Oc...


In [ ]:
# cross map the rows of filtered_smiley_data on the column Smiles_column_name, and for each and create a new df with new col names smiley_1, smiley_2, anf for all other columns, column name should be (lets say it is col_name) col_name_1, col_name_2 an col_name_1 should have the value of the row of filtered_smiley_data where the Smiles_column_name matched with smiley_1.



In [13]:
import pandas as pd
from numpy import nan

# Load CSV files
neuro_smiley_data = pd.read_csv(neuro_smiley_csv)
filtered_smiley_data = pd.read_csv(output_csv)

# Keep only necessary columns
neuro_cols = list(neuro_smiley_data.columns)
neuro_cols.remove(neuro_smile_col)
neuro_cols.remove(neuro_drug_name_col)
neuro_cols.remove(neuro_target_col)
neuro_smiley_data = neuro_smiley_data.drop(columns=neuro_cols)

# Clean and format text columns
neuro_smiley_data[neuro_smile_col] = neuro_smiley_data[neuro_smile_col].str.lstrip('\n')
neuro_smiley_data[neuro_drug_name_col] = neuro_smiley_data[neuro_drug_name_col].str.upper()
neuro_smiley_data[neuro_target_col] = neuro_smiley_data[neuro_target_col].str.upper()

# Explode targets for each smiley
neuro_smiley_data[neuro_target_col] = neuro_smiley_data[neuro_target_col].str.split(';')
neuro_smiley_data = neuro_smiley_data.explode(neuro_target_col)

# Extract all column names except Smiles_column_name
other_columns = [col for col in filtered_smiley_data.columns if col != Smiles_column_name]

# Create mappings for fast lookup
smiley_to_drug = neuro_smiley_data.set_index(neuro_smile_col)[neuro_drug_name_col].to_dict()
smiley_to_targets = neuro_smiley_data.groupby(neuro_smile_col)[neuro_target_col].apply(set).to_dict()

# Store filtered_smiley_data rows as a dictionary for fast lookup
smiley_to_row = filtered_smiley_data.set_index(Smiles_column_name).to_dict(orient="index")

# Get all unique smiley structures
smiley_list = filtered_smiley_data[Smiles_column_name].tolist()


In [14]:
cross_pairs = []
total_smileys = len(smiley_list)
q1 = total_smileys // 4  # Quarter 1 limit

# Generate first quarter of pairs
for i in range(q1):
    for j in range(i + 1, total_smileys):
        cross_pairs.append((smiley_list[i], smiley_list[j]))

# Process the first quarter
new_data = []

for smiley_1, smiley_2 in cross_pairs:
    row_1 = smiley_to_row[smiley_1]
    row_2 = smiley_to_row[smiley_2]

    new_row = {}

    drug_1 = smiley_to_drug.get(smiley_1, "UNKNOWN")
    drug_2 = smiley_to_drug.get(smiley_2, "UNKNOWN")

    drug_1, drug_2 = sorted([drug_1, drug_2])

    new_row["drug_1"] = drug_1
    new_row["drug_2"] = drug_2

    targets1 = smiley_to_targets.get(smiley_1, set())
    targets2 = smiley_to_targets.get(smiley_2, set())

    common_targets = targets1.intersection(targets2)
    common_targets.discard(nan)

    new_row["matched"] = bool(common_targets)
    new_row["common_targets"] = common_targets
    new_row["smiley_1"] = smiley_1
    new_row["smiley_2"] = smiley_2    

    for col in other_columns:
        new_row[f"{col}_1"] = row_1[col]
        new_row[f"{col}_2"] = row_2[col]

    new_data.append(new_row)

# Convert to DataFrame and save
cross_mapped_df = pd.DataFrame(new_data)
cross_mapped_df.to_csv("cross_mapped_part1.csv", index=False)

In [15]:
# value count common targets
common_target_counts = cross_mapped_df["common_targets"].value_counts()
common_target_counts 

common_targets
{}                                               6967
{ TP53}                                           424
{ CYP3A4}                                         139
{ FLT3}                                           106
{ ERBB2}                                          101
                                                 ... 
{ HRAS,  JAK2}                                      1
{ NRAS,  KIT,  JAK2}                                1
{ KRAS,  PTEN,  PIK3CA,  NRAS,  HRAS}               1
{ KRAS,  PTEN,  PIK3CA,  NRAS,  MAP2K1,  ATM}       1
{ APC,  NF1,  PTEN,  PIK3CA,  NRAS,  HRAS}          1
Name: count, Length: 1498, dtype: int64

In [16]:
cross_pairs = []

# Generate second quarter of pairs
for i in range(q1, 2 * q1):
    for j in range(i + 1, total_smileys):
        cross_pairs.append((smiley_list[i], smiley_list[j]))

# Process the second quarter
new_data = []

for smiley_1, smiley_2 in cross_pairs:
    row_1 = smiley_to_row[smiley_1]
    row_2 = smiley_to_row[smiley_2]

    new_row = {}

    drug_1 = smiley_to_drug.get(smiley_1, "UNKNOWN")
    drug_2 = smiley_to_drug.get(smiley_2, "UNKNOWN")

    drug_1, drug_2 = sorted([drug_1, drug_2])

    new_row["drug_1"] = drug_1
    new_row["drug_2"] = drug_2

    targets1 = smiley_to_targets.get(smiley_1, set())
    targets2 = smiley_to_targets.get(smiley_2, set())

    common_targets = targets1.intersection(targets2)
    common_targets.discard(nan)

    new_row["matched"] = bool(common_targets)
    new_row["common_targets"] = common_targets
    new_row["smiley_1"] = smiley_1
    new_row["smiley_2"] = smiley_2    

    for col in other_columns:
        new_row[f"{col}_1"] = row_1[col]
        new_row[f"{col}_2"] = row_2[col]

    new_data.append(new_row)

# Convert to DataFrame and save
cross_mapped_df = pd.DataFrame(new_data)
cross_mapped_df.to_csv("cross_mapped_part2.csv", index=False)


In [17]:
cross_pairs = []

# Generate third quarter of pairs
for i in range(2 * q1, 3 * q1):
    for j in range(i + 1, total_smileys):
        cross_pairs.append((smiley_list[i], smiley_list[j]))

# Process the third quarter
new_data = []

for smiley_1, smiley_2 in cross_pairs:
    row_1 = smiley_to_row[smiley_1]
    row_2 = smiley_to_row[smiley_2]

    new_row = {}

    drug_1 = smiley_to_drug.get(smiley_1, "UNKNOWN")
    drug_2 = smiley_to_drug.get(smiley_2, "UNKNOWN")

    drug_1, drug_2 = sorted([drug_1, drug_2])

    new_row["drug_1"] = drug_1
    new_row["drug_2"] = drug_2

    targets1 = smiley_to_targets.get(smiley_1, set())
    targets2 = smiley_to_targets.get(smiley_2, set())

    common_targets = targets1.intersection(targets2)
    common_targets.discard(nan)

    new_row["matched"] = bool(common_targets)
    new_row["common_targets"] = common_targets
    new_row["smiley_1"] = smiley_1
    new_row["smiley_2"] = smiley_2    

    for col in other_columns:
        new_row[f"{col}_1"] = row_1[col]
        new_row[f"{col}_2"] = row_2[col]

    new_data.append(new_row)

# Convert to DataFrame and save
cross_mapped_df = pd.DataFrame(new_data)
cross_mapped_df.to_csv("cross_mapped_part3.csv", index=False)


In [18]:
cross_pairs = []

# Generate fourth quarter of pairs
for i in range(3 * q1, total_smileys):
    for j in range(i + 1, total_smileys):
        cross_pairs.append((smiley_list[i], smiley_list[j]))

# Process the fourth quarter
new_data = []

for smiley_1, smiley_2 in cross_pairs:
    row_1 = smiley_to_row[smiley_1]
    row_2 = smiley_to_row[smiley_2]

    new_row = {}

    drug_1 = smiley_to_drug.get(smiley_1, "UNKNOWN")
    drug_2 = smiley_to_drug.get(smiley_2, "UNKNOWN")

    drug_1, drug_2 = sorted([drug_1, drug_2])

    new_row["drug_1"] = drug_1
    new_row["drug_2"] = drug_2

    targets1 = smiley_to_targets.get(smiley_1, set())
    targets2 = smiley_to_targets.get(smiley_2, set())

    common_targets = targets1.intersection(targets2)
    common_targets.discard(nan)

    new_row["matched"] = bool(common_targets)
    new_row["common_targets"] = common_targets
    new_row["smiley_1"] = smiley_1
    new_row["smiley_2"] = smiley_2    

    for col in other_columns:
        new_row[f"{col}_1"] = row_1[col]
        new_row[f"{col}_2"] = row_2[col]

    new_data.append(new_row)

# Convert to DataFrame and save
cross_mapped_df = pd.DataFrame(new_data)
cross_mapped_df.to_csv("cross_mapped_part4.csv", index=False)

In [19]:
# Load and merge all parts
df1 = pd.read_csv("cross_mapped_part1.csv")
df2 = pd.read_csv("cross_mapped_part2.csv")
df3 = pd.read_csv("cross_mapped_part3.csv")
df4 = pd.read_csv("cross_mapped_part4.csv")

final_df = pd.concat([df1, df2, df3, df4], ignore_index=True)
final_df.to_csv(cross_mapped_csv, index=False)


In [21]:
final_df["matched"].value_counts()

matched
False    15797
True      8513
Name: count, dtype: int64

In [22]:
final_df["matched"].value_counts()

matched
False    15797
True      8513
Name: count, dtype: int64

In [20]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import DataStructs
from rdkit.Chem.rdFingerprintGenerator import (
    GetMorganGenerator, GetRDKitFPGenerator, GetTopologicalTorsionGenerator, GetAtomPairGenerator
)
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem import rdFMCS
from tqdm import tqdm

# Load dataset
df = pd.read_csv("cancer_final_data.csv")

# Ensure missing values are handled
df = df.dropna(subset=["smiley_1", "smiley_2"])  # Drop rows with missing SMILES

# Fingerprint cache
fingerprint_cache = {}

def get_fingerprint(smiles, fp_type):
    """Generate molecular fingerprints based on type with caching."""
    if (smiles, fp_type) in fingerprint_cache:
        return fingerprint_cache[(smiles, fp_type)]
    
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    if fp_type == 'Morgan':
        fp = GetMorganGenerator(radius=2, fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'FeatMorgan':
        fp = GetMorganGenerator(radius=2, fpSize=2048, includeChirality=True).GetFingerprint(mol)
    elif fp_type == 'AtomPair':
        fp = GetAtomPairGenerator(fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'RDKit':
        fp = GetRDKitFPGenerator(fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'Torsion':
        fp = GetTopologicalTorsionGenerator(fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'Layered':
        fp = RDKFingerprint(mol)
    elif fp_type == 'MACCS':
        fp = rdMolDescriptors.GetMACCSKeysFingerprint(mol)
    else:
        raise ValueError(f"Unknown fingerprint type: {fp_type}")
    
    fingerprint_cache[(smiles, fp_type)] = fp
    return fp

def tanimoto_similarity(smiles1, smiles2, fp_type):
    """Compute Tanimoto similarity between two molecules."""
    fp1 = get_fingerprint(smiles1, fp_type)
    fp2 = get_fingerprint(smiles2, fp_type)

    if fp1 is None or fp2 is None:
        return None
    return DataStructs.FingerprintSimilarity(fp1, fp2)

def compute_tanimoto(fp_type):
    """Sequential computation of Tanimoto similarity."""
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Computing Tanimoto {fp_type}"):
        results.append(tanimoto_similarity(row["smiley_1"], row["smiley_2"], fp_type))
    return results

def compute_mcs(smiles1, smiles2):
    """Compute Maximum Common Substructure (MCS) similarity features."""
    mol1, mol2 = Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)
    if mol1 is None or mol2 is None:
        return None, None, None

    mcs_result = rdFMCS.FindMCS([mol1, mol2])
    mcs_size = mcs_result.numAtoms
    mcs_tanimoto = mcs_size / min(mol1.GetNumAtoms(), mol2.GetNumAtoms())
    mcs_overlap = mcs_size / max(mol1.GetNumAtoms(), mol2.GetNumAtoms())

    return mcs_size, mcs_tanimoto, mcs_overlap

def compute_mcs_features():
    """Sequential computation of MCS features."""
    mcs_sizes, mcs_tanimotos, mcs_overlaps = [], [], []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Computing MCS Features"):
        mcs_size, mcs_tanimoto, mcs_overlap = compute_mcs(row["smiley_1"], row["smiley_2"])
        mcs_sizes.append(mcs_size)
        mcs_tanimotos.append(mcs_tanimoto)
        mcs_overlaps.append(mcs_overlap)
    return mcs_sizes, mcs_tanimotos, mcs_overlaps

# Compute Tanimoto similarities for all fingerprint types
fp_types = ['Morgan', 'FeatMorgan', 'AtomPair', 'RDKit', 'Torsion', 'Layered', 'MACCS']
for fp in fp_types:
    df[f"Tanimoto_{fp}"] = compute_tanimoto(fp)

# Compute MCS Features
# df["MCS_Size"], df["MCS_Tanimoto"], df["MCS_Overlap"] = compute_mcs_features()

# Save updated dataframe
df.to_csv("cross_mapped_data_with_tanimoto_cancer.csv", index=False)
print("Feature extraction complete!")

# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.feature_selection import mutual_info_classif, RFE, SelectFromModel, SelectKBest, f_classif
# from sklearn.linear_model import LogisticRegression, Lasso
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, roc_auc_score

# # Load dataset
# df = pd.read_csv("cross_mapped_data_with_tanimoto_paper.csv")  # Updated filename

# # Extract feature pairs
# feature_pairs = {}
# for col in df.columns:
#     if "_" in col and col not in ["smiley_1", "smiley_2", "drug_1", "drug_2", "matched"]:
#         base = "_".join(col.split("_")[:-1])  # Generalized base feature extraction
#         if base not in feature_pairs:
#             feature_pairs[base] = []
#         feature_pairs[base].append(col)

# # Ensure pairs are correctly grouped
# paired_features = [pair for pair in feature_pairs.values() if len(pair) == 2]
# flattened_features = [feat for pair in paired_features for feat in pair]

# # Add new Tanimoto and MCS features
# additional_features = [
#     "Tanimoto_Morgan", "Tanimoto_FeatMorgan", "Tanimoto_AtomPair",
#     "Tanimoto_RDKit", "Tanimoto_Torsion", "Tanimoto_Layered", "Tanimoto_MACCS",
#     "MCS_Size", "MCS_Tanimoto", "MCS_Overlap"
# ]
# flattened_features.extend(additional_features)

# # Define X and y
# X = df[flattened_features]
# y = df["matched"]

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ### Feature Selection Methods
# # 1. Mutual Information
# mi_scores = mutual_info_classif(X_train, y_train)
# mi_scores_dict = {feat: mi for feat, mi in zip(flattened_features, mi_scores)}
# selected_mi = sorted(mi_scores_dict, key=mi_scores_dict.get, reverse=True)[:20]

# # 2. RFE with Logistic Regression
# log_reg = LogisticRegression(max_iter=1000)
# rfe = RFE(log_reg, n_features_to_select=20)
# rfe.fit(X_train, y_train)
# selected_rfe = [feat for feat, keep in zip(flattened_features, rfe.support_) if keep]

# # 3. Random Forest Feature Importance
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)
# rf_importances = {feat: imp for feat, imp in zip(flattened_features, rf.feature_importances_)}
# selected_rf = sorted(rf_importances, key=rf_importances.get, reverse=True)[:20]

# # 4. Lasso Regression (L1 Regularization)
# lasso = Lasso(alpha=0.01)
# lasso.fit(X_train, y_train)
# lasso_coeffs = {feat: coef for feat, coef in zip(flattened_features, lasso.coef_)}
# selected_lasso = [feat for feat, coef in lasso_coeffs.items() if abs(coef) > 0]

# # 5. XGBoost Feature Importance
# xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
# xgb.fit(X_train, y_train)
# xgb_importances = {feat: imp for feat, imp in zip(flattened_features, xgb.feature_importances_)}
# selected_xgb = sorted(xgb_importances, key=xgb_importances.get, reverse=True)[:20]

# # 6. SelectKBest (ANOVA F-score)
# kbest = SelectKBest(score_func=f_classif, k=20)
# kbest.fit(X_train, y_train)
# selected_kbest = [feat for feat, keep in zip(flattened_features, kbest.get_support()) if keep]

# ### Ensure features are kept in pairs
# def enforce_feature_pairs(selected_features):
#     selected_pairs = []
#     for pair in paired_features:
#         if any(f in selected_features for f in pair):
#             selected_pairs.extend(pair)
#     return list(set(selected_pairs + [f for f in selected_features if f in additional_features]))

# selected_mi_pairs = enforce_feature_pairs(selected_mi)
# selected_rfe_pairs = enforce_feature_pairs(selected_rfe)
# selected_rf_pairs = enforce_feature_pairs(selected_rf)
# selected_lasso_pairs = enforce_feature_pairs(selected_lasso)
# selected_xgb_pairs = enforce_feature_pairs(selected_xgb)
# selected_kbest_pairs = enforce_feature_pairs(selected_kbest)

# ### Train Models with Selected Features
# def train_model(X_train, X_test, y_train, y_test, features, model):
#     X_train_selected = X_train[features]
#     X_test_selected = X_test[features]
#     model.fit(X_train_selected, y_train)
#     y_pred = model.predict(X_test_selected)
#     y_pred_proba = model.predict_proba(X_test_selected)[:, 1] if hasattr(model, "predict_proba") else y_pred
#     return accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred_proba)

# models = {
#     "Logistic Regression": LogisticRegression(max_iter=1000),
#     "SVM": SVC(probability=True),  # Enable probability for AUC calculation
#     "Random Forest": RandomForestClassifier(n_estimators=100),
#     "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
# }

# accuracy_results = {}
# auc_results = {}

# for name, model in models.items():
#     accuracy_results[name] = {}
#     auc_results[name] = {}
#     for method, selected_features in {
#         "Mutual Information": selected_mi_pairs,
#         "RFE": selected_rfe_pairs,
#         "Random Forest": selected_rf_pairs,
#         "Lasso": selected_lasso_pairs,
#         "XGBoost": selected_xgb_pairs,
#         "SelectKBest": selected_kbest_pairs,
#     }.items():
#         acc, auc = train_model(X_train, X_test, y_train, y_test, selected_features, model)
#         accuracy_results[name][method] = acc
#         auc_results[name][method] = auc

# ### Print Results
# print("\nFeature Selection Results (Top Selected Features in Pairs):")
# print("Mutual Information:", selected_mi_pairs)
# print("RFE:", selected_rfe_pairs)
# print("Random Forest:", selected_rf_pairs)
# print("Lasso:", selected_lasso_pairs)
# print("XGBoost:", selected_xgb_pairs)
# print("SelectKBest:", selected_kbest_pairs)

# print("\nModel Performance with Selected Features:")
# for model in models.keys():
#     print(f"\n{model}:")
#     for method in accuracy_results[model].keys():
#         print(f"  {method}: Accuracy = {accuracy_results[model][method]:.4f}, AUC = {auc_results[model][method]:.4f}")


Computing Tanimoto FeatMorgan:   0%|          | 0/24310 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [31]:
df.head()

,drug_1,drug_2,matched,common_targets,smiley_1,smiley_2,ABC_1,ABC_2,ABCGG_1,ABCGG_2,...,mZagreb1_2,mZagreb2_1,mZagreb2_2,Tanimoto_Morgan,Tanimoto_FeatMorgan,Tanimoto_AtomPair,Tanimoto_RDKit,Tanimoto_Torsion,Tanimoto_Layered,Tanimoto_MACCS
0,ABEMACICLIB,ABIRATERONE,False,set(),CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,C[C@]12CC[C@H](O)CC1=CC[C@@H]1[C@@H]2CC[C@]2(C...,29.255246,21.272672,20.627212,16.066691,...,7.402778,8.027778,5.416667,0.089109,0.085714,0.250441,0.299145,0.055118,0.299145,0.309859
1,ABEMACICLIB,ACALABRUTINIB,False,set(),CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,CC#CC(=O)N1CCC[C@H]1c1nc(-c2ccc(C(=O)Nc3ccccn3...,29.255246,27.541373,20.627212,20.914869,...,10.083333,8.027778,7.750000,0.120690,0.120690,0.382911,0.523977,0.196581,0.523977,0.575758
2,ABEMACICLIB,ACLARUBICIN,False,set(),CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,CC[C@@]1(O)C[C@H](O[C@H]2C[C@H](N(C)C)[C@H](O[...,29.255246,45.896458,20.627212,32.886584,...,22.090278,8.027778,12.444444,0.079710,0.082759,0.289040,0.529257,0.076923,0.529257,0.395062
3,ABEMACICLIB,ADAGRASIB,False,set(),CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,C=C(F)C(=O)N1CCN(c2nc(OC[C@@H]3CCCN3C)nc3c2CCN...,29.255246,34.043233,20.627212,24.666546,...,13.027778,8.027778,9.388889,0.097744,0.097744,0.343342,0.513245,0.152672,0.513245,0.636364
4,ABEMACICLIB,AFATINIB,False,set(),CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,29.255246,26.548139,20.627212,20.094967,...,10.722222,8.027778,7.444444,0.132231,0.141667,0.403279,0.441804,0.145455,0.441804,0.543210


In [1]:
import pandas as pd
# df = pd.read_csv("paper_final_data.csv", )
df = pd.read_csv("paper_final_data.csv", usecols=["smiley_1", "smiley_2"])

In [2]:
print(len(df))

979300


In [3]:
count = 0
for _, row in df.iterrows():
    count += 1

print("Rows processed:", count)  # Should match len(df) = 979300


Rows processed: 979300


In [6]:
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdFingerprintGenerator import (
    GetMorganGenerator, GetRDKitFPGenerator, GetTopologicalTorsionGenerator, GetAtomPairGenerator
)
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from joblib import Parallel, delayed
from tqdm import tqdm
from functools import lru_cache

# Convert SMILES columns to NumPy arrays (faster access)
smiles1_arr = df["smiley_1"].to_numpy()
smiles2_arr = df["smiley_2"].to_numpy()

# Constants
NUM_CORES = 7  # Adjust based on available CPU cores
CHUNK_SIZE = 500  # Batch processing size

@lru_cache(maxsize=10000)
def get_fingerprint(smiles, fp_type):
    """Generate molecular fingerprints with caching."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    if fp_type == 'Morgan':
        return GetMorganGenerator(radius=2, fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'FeatMorgan':
        return GetMorganGenerator(radius=2, fpSize=2048, includeChirality=True).GetFingerprint(mol)
    elif fp_type == 'AtomPair':
        return GetAtomPairGenerator(fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'RDKit':
        return GetRDKitFPGenerator(fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'Torsion':
        return GetTopologicalTorsionGenerator(fpSize=2048).GetFingerprint(mol)
    elif fp_type == 'Layered':
        return RDKFingerprint(mol)
    elif fp_type == 'MACCS':
        return rdMolDescriptors.GetMACCSKeysFingerprint(mol)
    else:
        return None  # Unknown fingerprint type

def compute_tanimoto(s1, s2, fp_type):
    """Compute Tanimoto similarity between two molecules."""
    fp1 = get_fingerprint(s1, fp_type)
    fp2 = get_fingerprint(s2, fp_type)

    if fp1 is None or fp2 is None:
        return 0.0  # Return 0 similarity if fingerprints are invalid
    
    return DataStructs.FingerprintSimilarity(fp1, fp2)

def parallel_tanimoto(fp_type):
    """Parallel computation of Tanimoto similarity using batch processing."""
    results = np.zeros(len(df), dtype=np.float32)

    for i in tqdm(range(0, len(df), CHUNK_SIZE), desc=f"Tanimoto {fp_type}"):
        end_idx = min(i + CHUNK_SIZE, len(df))  # Ensure last chunk is handled correctly

        batch_results = Parallel(n_jobs=NUM_CORES, backend="multiprocessing")(
            delayed(compute_tanimoto)(s1, s2, fp_type)
            for s1, s2 in zip(smiles1_arr[i:end_idx], smiles2_arr[i:end_idx])
        )

        results[i:end_idx] = batch_results

    return results

def compute_mcs(smiles1, smiles2):
    """Compute Maximum Common Substructure (MCS) similarity features."""
    mol1, mol2 = Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)
    if mol1 is None or mol2 is None:
        return 0, 0.0, 0.0

    num_atoms = min(mol1.GetNumAtoms(), mol2.GetNumAtoms())
    timeout = 0.5 if num_atoms < 20 else (1.5 if num_atoms < 50 else 2.0)

    try:
        mcs_result = rdFMCS.FindMCS(mols=[mol1, mol2], maximizeBonds=True, timeout=int(timeout * 1000))
        mcs_size = mcs_result.numAtoms
        mcs_tanimoto = mcs_size / min(mol1.GetNumAtoms(), mol2.GetNumAtoms()) if mcs_size else 0.0
        mcs_overlap = mcs_size / max(mol1.GetNumAtoms(), mol2.GetNumAtoms()) if mcs_size else 0.0
    except Exception as e:
        print(f"Error computing MCS for {smiles1} and {smiles2}: {e}")
        return 0, 0.0, 0.0

    return mcs_size, mcs_tanimoto, mcs_overlap

def parallel_mcs():
    """Parallel computation of MCS features using batch processing."""
    mcs_sizes = np.zeros(len(df), dtype=np.float32)
    mcs_tanimotos = np.zeros(len(df), dtype=np.float32)
    mcs_overlaps = np.zeros(len(df), dtype=np.float32)

    for i in tqdm(range(0, len(df), CHUNK_SIZE), desc="MCS Computation"):
        end_idx = min(i + CHUNK_SIZE, len(df))  # Ensure last chunk is handled correctly

        batch_results = Parallel(n_jobs=NUM_CORES)(
            delayed(compute_mcs)(s1, s2) for s1, s2 in zip(smiles1_arr[i:end_idx], smiles2_arr[i:end_idx])
        )
        sizes, tanimotos, overlaps = zip(*batch_results)

        mcs_sizes[i:end_idx] = sizes
        mcs_tanimotos[i:end_idx] = tanimotos
        mcs_overlaps[i:end_idx] = overlaps

    return mcs_sizes, mcs_tanimotos, mcs_overlaps

# Compute Tanimoto similarities in parallel
# fp_types = ['Morgan', 'FeatMorgan', 'AtomPair', 'RDKit', 'Torsion', 'Layered', 'MACCS']
# for fp in fp_types:
#     df[f"Tanimoto_{fp}"] = parallel_tanimoto(fp)

# Compute MCS Features in parallel
df["MCS_Size"], df["MCS_Tanimoto"], df["MCS_Overlap"] = parallel_mcs()

# Save updated dataframe
df.to_csv("smiley_mcs_paper.csv", index=False)
print("✅ Feature extraction complete!")


MCS Computation:   0%|          | 0/1959 [00:00<?, ?it/s]

MCS Computation:   3%|▎         | 54/1959 [06:24<3:46:10,  7.12s/it]


KeyboardInterrupt: 

In [3]:
df.head()

,smiley_1,smiley_2,Tanimoto_Morgan,Tanimoto_FeatMorgan,Tanimoto_AtomPair,Tanimoto_RDKit,Tanimoto_Torsion,Tanimoto_Layered,Tanimoto_MACCS
0,C[S+](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H]([C...,CC1=CC(=CC=C1)NC2=C(C=NC=C2)S(=O)(=O)NC(=O)NC(C)C,0.085106,0.083333,0.172727,0.307380,0.010870,0.307380,0.316327
1,C[S+](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H]([C...,C1=CC(=C2C(=C1NCCNCCO)C(=O)C3=C(C=CC(=C3C2=O)O...,0.061728,0.060241,0.166355,0.329656,0.010753,0.329656,0.506329
2,C[S+](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H]([C...,CCOC(=O)C1=C2CN(C(=O)C3=C(N2C=N1)C=CC(=C3)F)C,0.130435,0.115789,0.197150,0.433962,0.131868,0.433962,0.554217
3,C[S+](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H]([C...,C1CCC(CC1)NC(=O)N(CCCl)N=O,0.062500,0.060976,0.108635,0.121653,0.041667,0.121653,0.344828
4,C[S+](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H]([C...,C1=CC(=CC(=C1)C(F)(F)F)/C(=N\OCCCCC(=O)O)/C2=C...,0.085106,0.082474,0.231293,0.243031,0.000000,0.243031,0.337209


In [4]:
print(df.columns)

Index(['smiley_1', 'smiley_2', 'Tanimoto_Morgan', 'Tanimoto_FeatMorgan',
       'Tanimoto_AtomPair', 'Tanimoto_RDKit', 'Tanimoto_Torsion',
       'Tanimoto_Layered', 'Tanimoto_MACCS'],
      dtype='object')


In [5]:
df.to_csv("tanimoto_smiley_paper.csv", index=False)

In [ ]:
import pandas as pd

# Load the CSV files
df1 = pd.read_csv("paper_final_data.csv")
df2 = pd.read_csv("tanimoto_smiley_paper.csv")

# Merge on smiley1 and smiley2
merged_df = pd.merge(df1, df2, on=["smiley_1", "smiley_2"], how="inner")  # Use "outer" for full join


: 

In [8]:
merged_df.to_csv("training_data_paper.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('training_data_paper.csv', low_memory=False)

In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif, RFE, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from joblib import Parallel, delayed

# ========== 1. Stratified Sampling ==========
df_sample = df

# ========== 2. Extract paired features ==========
feature_pairs = {}
for col in df_sample.columns:
    if "_" in col and col not in ["smiley_1", "smiley_2", "drug_1", "drug_2", "matched", "common_targets"]:
        base = "_".join(col.split("_")[:-1])
        feature_pairs.setdefault(base, []).append(col)

paired_features = [pair for pair in feature_pairs.values() if len(pair) == 2]
flattened_features = [f for pair in paired_features for f in pair]

additional_features = [
    "Tanimoto_Morgan", "Tanimoto_FeatMorgan", "Tanimoto_AtomPair",
    "Tanimoto_RDKit", "Tanimoto_Torsion", "Tanimoto_Layered", "Tanimoto_MACCS"
]
flattened_features.extend(additional_features)

# ========== 3. Prepare X and y ==========
X = df_sample[flattened_features]
y = df_sample["matched"].values

# ========== 4. Scale ==========
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ========== 5. Train-test split ==========
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# ========== 6. Feature Pair Enforcer ==========
def enforce_feature_pairs(selected_features):
    selected_pairs = []
    for pair in paired_features:
        if any(f in selected_features for f in pair):
            selected_pairs.extend(pair)
    return list(set(selected_pairs + [f for f in selected_features if f in additional_features]))

# ========== 7. Feature Selection Methods ==========
def run_feature_selection(method):
    try:
        if method == "Mutual Information":
            scores = mutual_info_classif(X_train, y_train)
            selected = sorted(zip(flattened_features, scores), key=lambda x: x[1], reverse=True)[:20]
            return method, enforce_feature_pairs([f for f, _ in selected])

        elif method == "RFE":
            rfe = RFE(LogisticRegression(max_iter=1000), n_features_to_select=20)
            rfe.fit(X_train, y_train)
            return method, enforce_feature_pairs([f for f, s in zip(flattened_features, rfe.support_) if s])

        elif method == "Random Forest":
            rf = RandomForestClassifier(n_estimators=100, random_state=42)
            rf.fit(X_train, y_train)
            scores = rf.feature_importances_
            selected = sorted(zip(flattened_features, scores), key=lambda x: x[1], reverse=True)[:20]
            return method, enforce_feature_pairs([f for f, _ in selected])

        elif method == "Lasso":
            lasso = Lasso(alpha=0.01)
            lasso.fit(X_train, y_train)
            return method, enforce_feature_pairs([f for f, c in zip(flattened_features, lasso.coef_) if abs(c) > 0])

        elif method == "XGBoost":
            xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
            xgb.fit(X_train, y_train)
            scores = xgb.feature_importances_
            selected = sorted(zip(flattened_features, scores), key=lambda x: x[1], reverse=True)[:20]
            return method, enforce_feature_pairs([f for f, _ in selected])

        elif method == "SelectKBest":
            skb = SelectKBest(score_func=f_classif, k=20).fit(X_train, y_train)
            return method, enforce_feature_pairs([f for f, s in zip(flattened_features, skb.get_support()) if s])

    except Exception as e:
        print(f"Error in {method}: {e}")
        return method, []

fs_methods = ["Mutual Information", "RFE", "Random Forest", "Lasso", "XGBoost", "SelectKBest"]

print("Running feature selection...")
results = Parallel(n_jobs=-1)(delayed(run_feature_selection)(method) for method in fs_methods)
selected_features = dict(results)

# ========== 8. Train Models ==========
def train_model(X_train, X_test, y_train, y_test, features, model):
    idx = [flattened_features.index(f) for f in features if f in flattened_features]
    X_train_sel = X_train[:, idx]
    X_test_sel = X_test[:, idx]

    model.fit(X_train_sel, y_train)
    y_pred = model.predict(X_test_sel)
    y_proba = model.predict_proba(X_test_sel)[:, 1] if hasattr(model, "predict_proba") else y_pred
    return accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_proba)

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(probability=True),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
}

accuracy_results, auc_results = {}, {}

print("Training models...")
for model_name, model in tqdm(models.items()):
    accuracy_results[model_name] = {}
    auc_results[model_name] = {}
    for method, features in selected_features.items():
        acc, auc = train_model(X_train, X_test, y_train, y_test, features, model)
        accuracy_results[model_name][method] = acc
        auc_results[model_name][method] = auc

# ========== 9. Print Results ==========
print("\nFeature Selection Summary:")
for method, features in selected_features.items():
    print(f"{method}: {features}")

print("\nModel Performance:")
for model in models:
    print(f"\n{model}:")
    for method in fs_methods:
        print(f"  {method}: Accuracy = {accuracy_results[model][method]:.4f}, AUC = {auc_results[model][method]:.4f}")


Running feature selection...


/home/prateek/miniconda3/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [1836 1837 1838 1839 1840 1841 1842 1843 1846 1847 1862 1863 1874 1875
 1878 1879 1888 1889 1900 1901 1912 1913 1914 1915 1916 1917 1918 1919
 1920 1921 1922 1923 1924 1925 1928 1929 1938 1939 1944 1945 1946 1947
 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961
 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1976 1977
 1978 1979 1980 1981 1982 1983 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998 1999 2000 2001 2004 2005 2020 2021 2032 2033
 2036 2037 2046 2047 2058 2059 2070 2071 2072 2073 2074 2075 2076 2077
 2078 2079 2080 2081 2082 2083 2086 2087 2096 2097 2102 2103 2104 2105
 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2116 2117 2118 2119
 2120 2121 2122 2123 2124 2125 2126 2127 2128 2129 2130 2131 2134 2135
 2136 2137 2138 2139 2140 2141 2144 2145 2146 2147 2148 2149 2150 2151
 2744 2745 2764 

KeyboardInterrupt: 